# Question 1:  Pose Graph Optimization for 1D SLAM

A solved example for 1D SLAM which optimizes for pose variables using weighted least squares method (Gauss Newton) has been explained in the class. You can view it at `examples/1D-SLAM.ipynb`. Your first task is to code this from scratch.

For this section, you have to calculate Jacobian analytically yourself and use it. However, you can check how correct it is using `jax`'s `jacobian`. Its usage is explained in the supplementary notebook (`examples/`).

## How to get started?

1. Go through the example and understand the optimization procedure.
2. Generate the data as in the example.
3. Write the derivatives of the function with respect to the constraints
4. Create the final jacobian and write the update steps using Gauss Newton

This tiny example will help make sense of the next question, as it's largely just an extension of this.

Finally, plot the loop closure constraint, poses, and error at every iteration as well. Since this is a one dimensional example, you can just assume x = y.

In [23]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [33]:
from optimizer import Optimizer

In [34]:
initial_values = np.array([0, 1.1, 2.1, 3.2, 0.5]).reshape(-1, 1)
f = lambda x: np.array(
    [x[0] + 1.1 - x[1], x[1] + 1 - x[2], x[2] + 1.1 - x[3], x[3] - 2.7 - x[4],
     x[0] + 0 - x[4], x[0] - 0]).reshape(-1, 1)
jac = lambda x: np.array(
    [[1, -1, 0, 0, 0], [0, 1, -1, 0, 0], [0, 0, 1, -1, 0], [0, 0, 0, 1, -1],
     [1, 0, 0, 0, -1], [1, 0, 0, 0, 0]])
info_mat = np.eye(6)
info_mat *= 100
info_mat[5, 5] = 1000

In [35]:
info_mat

array([[ 100.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  100.,    0.,    0.,    0.,    0.],
       [   0.,    0.,  100.,    0.,    0.,    0.],
       [   0.,    0.,    0.,  100.,    0.,    0.],
       [   0.,    0.,    0.,    0.,  100.,    0.],
       [   0.,    0.,    0.,    0.,    0., 1000.]])

In [36]:
solver = Optimizer(f, jac, initial_values, info_mat, 1, algo='LM')
# n_iter only 1 because its linear


INFO:Optimizer:Using ALGO: LM
INFO:Optimizer:Initial Error: 12.5


In [38]:
solver.optimize()

DEBUG:Optimizer:LM: Update Accepted


Iteration: 1 / 1
Loss: 12.5
Iteration: 2 / 1
Loss: 2.5000000037997787


In [39]:
solver.get_current()

array([[9.99970001e-07],
       [1.00000500e+00],
       [1.90000800e+00],
       [2.90000900e+00],
       [1.00007000e-01]])

In [40]:
solver.print_info()

Iteration: 2 / 1
Loss: 2.5000000037997787


In [41]:
solver.__dict__

{'f': <function __main__.<lambda>(x)>,
 'jac': <function __main__.<lambda>(x)>,
 'initial_values': array([[0. ],
        [1.1],
        [2.1],
        [3.2],
        [0.5]]),
 'info_mat': array([[ 100.,    0.,    0.,    0.,    0.,    0.],
        [   0.,  100.,    0.,    0.,    0.,    0.],
        [   0.,    0.,  100.,    0.,    0.,    0.],
        [   0.,    0.,    0.,  100.,    0.,    0.],
        [   0.,    0.,    0.,    0.,  100.,    0.],
        [   0.,    0.,    0.,    0.,    0., 1000.]]),
 'n_iter': 1,
 'algo': 'LM',
 'lm_lambda': 0.0001,
 'F': <function optimizer.Optimizer.__init__.<locals>.<lambda>(x)>,
 '_Optimizer__current': array([[9.99970001e-07],
        [1.00000500e+00],
        [1.90000800e+00],
        [2.90000900e+00],
        [1.00007000e-01]]),
 '_Optimizer__iter': 2}

In [12]:
solver.__current

AttributeError: 'Optimizer' object has no attribute '__current'

In [1]:
def LM(J,info,residual,lamb):
    b = -jnp.matmul(jnp.matmul((J.T),(info)),jnp.array(residual))
    H = jnp.matmul(jnp.matmul((J.T),(info)),J)
    H = H + lamb * jnp.eye(H.shape[0])
    deltaX = jnp.matmul(jnp.linalg.pinv(H),b)
    return deltaX

In [ ]:
LM(jac(ini